#**Código nacional**
Esto es una prueba. Esto será el código para tratar todo nacional.

##Paquetes necesarios

In [1]:
#INSTALACIÓN DE PAQUETES NECESARIOS

In [4]:
!pip install geopandas
!pip install dbfread

In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF

##Paso dos - Información censal

In [6]:
#DATOS PARA PASO DOS

#INSERTAR MUN NECESARIOS
entidad = ['11']
municipios = ['007', '011', '044']

#RUTAS ENTRADA
MZNA_2020_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_11CSV20.csv"
ITER_2020_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\ITER\ITER_NALCSV20.csv"
MZNA_2010_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_11DBF10.dbf"
ITER_2010_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\ITER\ITER_NALDBF10.dbf"

#RUTAS SALIDA
MZNA_2020_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\MZNA_2020_CYA.csv"
ITER_2020_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\MZNA_2010_CYA.csv"
MZNA_2010_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\ITER_2020_CYA.csv"
ITER_2010_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\ITER_2010_CYA.csv"


In [7]:
#PASO DOS - Preparación de información censal

# Función para cargar datos de archivo DBF y convertirlos a DataFrame
def load_dbf_to_dataframe(file_path):
    table = DBF(file_path)
    return pd.DataFrame(iter(table))

# Cargar archivos en un DataFrame
MZNA_2020_df = pd.read_csv(MZNA_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'AGEB': str, 'MZA': str})
ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'LATITUD': str})

MZNA_2010_df = load_dbf_to_dataframe(MZNA_2010_ruta)
ITER_2010_df = load_dbf_to_dataframe(ITER_2010_ruta)

# 2.2 Homologación de ID urbana - CVEGEO
def homologate_urban_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}{row['AGEB']}{row['MZA']}"

# 3.2 Homologación de ID rural - CVEGEO
def homologate_rural_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}"

MZNA_2020_df['CVEGEO'] = MZNA_2020_df.apply(homologate_urban_id, axis=1)
MZNA_2010_df['CVEGEO'] = MZNA_2010_df.apply(homologate_urban_id, axis=1)

ITER_2020_df['CVEGEO'] = ITER_2020_df.apply(homologate_rural_id, axis=1)
ITER_2010_df['CVEGEO'] = ITER_2010_df.apply(homologate_rural_id, axis=1)

#reemplazar * con 0
def replace(dataframe):
    return dataframe.replace('*', '0')

MZNA_2020_df_rep = replace(MZNA_2020_df)
MZNA_2010_df_rep = replace(MZNA_2010_df)
ITER_2020_df_rep = replace(ITER_2020_df)
ITER_2010_df_rep = replace(ITER_2010_df)

# Función para filtrar información por municipios correspondientes
def filtrar_municipios(dataframe, municipios):
    return dataframe[dataframe['MUN'].isin(municipios)]

# aplicar filtrado por municipios
MZNA_2020_df_filter = filtrar_municipios(MZNA_2020_df_rep, municipios)
MZNA_2010_df_filter = filtrar_municipios(MZNA_2010_df_rep, municipios)

#Filtrar ITER
def filtrar_municipios_iter(dataframe, municipios, entidad):
    return dataframe[(dataframe['MUN'].isin(municipios)) & (dataframe['ENTIDAD'].isin(entidad))]

ITER_2020_df_filter = filtrar_municipios_iter(ITER_2020_df_rep, municipios, entidad)
ITER_2010_df_filter = filtrar_municipios_iter(ITER_2010_df_rep, municipios, entidad)

# 2.3 Guardado de información en CSV
def save_to_csv(dataframe, filename):
    dataframe.to_csv(filename, index=False)

save_to_csv(MZNA_2020_df_filter, MZNA_2020_salida)
save_to_csv(MZNA_2010_df_filter, ITER_2020_salida)
save_to_csv(ITER_2020_df_filter, MZNA_2010_salida)
save_to_csv(ITER_2010_df_filter, ITER_2010_salida)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Alejandro\\Downloads\\DATA\\2020\\MZNA\\RESAGEBURB_11CSV20.csv'

## Pasos tres a seis: Información geo-estadística

In [ ]:
#DATOS PARA PASO TRES A SEIS

#rutas shp base
SHP_MZN_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\11m.shp"
SHP_LOC_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\11l.shp"

#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\Mapa_Tipologías_CYA"

# Definir área de estudio
entidad = ['11']
municipios = ['007', '011', '044']

#Valor del filtro
filtro = 0.125

In [ ]:
#PASO TRES - Unión de información estadística y cartografía

#Carga de capas de manzanas y localidades a gdf
SHP_MZN_gdf = gpd.read_file(SHP_MZN_ruta)
SHP_LOC_gdf = gpd.read_file(SHP_LOC_ruta)

#Filtrar ITER
def filtrar_municipios_gpkg(dataframe, municipios, entidad):
    return dataframe[(dataframe['CVE_MUN'].isin(municipios)) & (dataframe['CVE_ENT'].isin(entidad))]

SHP_MZN_gdf_filter = filtrar_municipios_gpkg(SHP_MZN_gdf, municipios, entidad)
SHP_LOC_gdf_filter = filtrar_municipios_gpkg(SHP_LOC_gdf, municipios, entidad)

# CREAR GPKG BASE DESDE gdf DE MANZANAS
SHP_MZN_gdf_filter.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver='GPKG')

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_gpkg, layer='Mapa_Tipologias')

# Selección de ITER rural
SHP_LOC_gdf_filter_RURAL = SHP_LOC_gdf_filter[SHP_LOC_gdf_filter['AMBITO'] == 'Rural']

# Concatenar ITER rural a la capa existente
gdf_concatenated = gpd.GeoDataFrame(pd.concat([MAPA_TIPO_gpkg_gdf, SHP_LOC_gdf_filter_RURAL], ignore_index=True))

# Guardar la capa concatenada en el GeoPackage base
gdf_concatenated.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver='GPKG', overwrite=True)

# Re-leer la capa del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

# Unir campos de los DataFrames al GeoPackage base por CVEGEO
MAPA_TIPO_gdf1 = MAPA_TIPO_gdf.merge(MZNA_2020_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2020', '_2020_MZNA'))
MAPA_TIPO_gdf2 = MAPA_TIPO_gdf1.merge(ITER_2020_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2020', '_2020_ITER'))
MAPA_TIPO_gdf3 = MAPA_TIPO_gdf2.merge(MZNA_2010_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2010', '_2010_MZNA'))
MAPA_TIPO_gdf4 = MAPA_TIPO_gdf3.merge(ITER_2010_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2010', '_2010_ITER'))

#limpiar  *
MAPA_TIPO_gdf4_A = replace(MAPA_TIPO_gdf4)

#reemplazar N/D
MAPA_TIPO_gdf4_B = MAPA_TIPO_gdf4_A.replace('N/D', np.nan)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_B.fillna(0)

# Crear campos POPTOT2010, POPTOT2020, VIVPAR2010 y VIVPAR2020 y llenado respectivamente
MAPA_TIPO_gdf4_C['POPTOT2010'] = MAPA_TIPO_gdf4_B['POBTOT_2010'].astype(int) + MAPA_TIPO_gdf4_B['POBTOT_2010_ITER'].astype(int)
MAPA_TIPO_gdf4_C['POPTOT2020'] = MAPA_TIPO_gdf4_B['POBTOT_2020'].astype(int) + MAPA_TIPO_gdf4_B['POBTOT_2020_ITER'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR2010'] = MAPA_TIPO_gdf4_B['TVIVPAR_2010'].astype(int) + MAPA_TIPO_gdf4_B['TVIVPAR_2010_ITER'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR2020'] = MAPA_TIPO_gdf4_B['TVIVPAR_2020'].astype(int) +  MAPA_TIPO_gdf4_B['TVIVPAR_2020_ITER'].astype(int)
MAPA_TIPO_gdf4_C['VIVPARDES2020'] = MAPA_TIPO_gdf4_B['VIVPAR_DES_2020'].astype(int) + MAPA_TIPO_gdf4_B['VIVPAR_DES_2020_ITER'].astype(int)

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf4_C.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver="GPKG")

# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

# Creación de campos y llenado de información
MAPA_TIPO_gdf['DIFPOP'] = MAPA_TIPO_gdf['POPTOT2020'] - MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['DIFVIV'] = MAPA_TIPO_gdf['VIVPAR2020'] - MAPA_TIPO_gdf['VIVPAR2010']
MAPA_TIPO_gdf['RELPOP'] = MAPA_TIPO_gdf['DIFPOP'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['RELVIV'] = MAPA_TIPO_gdf['DIFVIV'] / MAPA_TIPO_gdf['VIVPAR2010']
MAPA_TIPO_gdf['CVEPOP'] = np.where(MAPA_TIPO_gdf['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_gdf['CVEVIV'] = np.select(
    [MAPA_TIPO_gdf['RELVIV'] < -0.075, (MAPA_TIPO_gdf['RELVIV'] >= -0.075) & (MAPA_TIPO_gdf['RELVIV'] <= 0.075), MAPA_TIPO_gdf['RELVIV'] > 0.075],
    [1, 2, 3]
)
MAPA_TIPO_gdf['CVE'] = MAPA_TIPO_gdf['CVEPOP'] + MAPA_TIPO_gdf['CVEVIV'].astype(str)
MAPA_TIPO_gdf['TIPOLOGIA'] = np.where((MAPA_TIPO_gdf['POPTOT2020'] == 0) & (MAPA_TIPO_gdf['POPTOT2010'] == 0) & (MAPA_TIPO_gdf['VIVPAR2010'] == 0) & (MAPA_TIPO_gdf['CVE'] == 'B3'), 'B3\'', MAPA_TIPO_gdf['CVE'])
MAPA_TIPO_gdf['area_ha'] = MAPA_TIPO_gdf.area / 10000
MAPA_TIPO_gdf['FILTRADO'] = MAPA_TIPO_gdf['VIVPAR2020'] / MAPA_TIPO_gdf['area_ha']**2

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver="GPKG")

# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

# Paso 6: Filtrado de información
MAPA_TIPO_gdf_FILTRO = MAPA_TIPO_gdf[MAPA_TIPO_gdf['FILTRADO'] >= filtro]

# Guardar el resultado final
MAPA_TIPO_gdf_FILTRO.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias_Filtrado', driver='GPKG')

##Paso siete: Representación gráfica

In [ ]:
#DATOS PARA GRÁFICA

# Ruta del mapa rápido
mapa_rapido_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\mapa_tipologias.png"

In [ ]:
# Paso 7: Representación gráfica rápida (desde código)
import matplotlib.pyplot as plt

def plot_map_with_legend(gdf, colormap, legend_labels, legend_title, save_path):
    fig, ax = plt.subplots(1, 1)

    # Mapear cada categoría a su color correspondiente
    for category, color in colormap.items():
        MAPA_TIPO_gdf_FILTRO[MAPA_TIPO_gdf_FILTRO['TIPOLOGIA'] == category].plot(ax=ax, color=color, label=category, edgecolor='none')

    # Crear leyenda
    handles = [plt.Rectangle((0,0),1,1, color=color) for color in colormap.values()]
    ax.legend(handles, legend_labels, title=legend_title)

    # Eliminar línea de contorno
    ax.axis('off')

    # Guardar el mapa como imagen
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

# Definir el diccionario de colores
colormap = {
    'DH': '#882A1C',
    'A1': '#E33F22',
    'A2': '#E5B636',
    'A3': '#FAF36A',
    'B1': '#CAEA60',
    'B2': '#60D512',
    'B3': '#39781C',
    'B3\'': '#254D12'
}

# Definir etiquetas para la leyenda
legend_labels = list(colormap.keys())

# Título de la leyenda
legend_title = 'Tipología'

# Ruta donde se guardará el mapa
map_image_path = mapa_rapido_ruta

# Llamar a la función para generar y guardar el mapa
plot_map_with_legend(MAPA_TIPO_gdf_FILTRO, colormap, legend_labels, legend_title, map_image_path)
